In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *


In [0]:
df = spark.sql('select * from sales_prj.gold.orders')
df.display()

In [0]:
spark.sql('select * from sales_prj.gold.customers').display()
spark.sql('select * from sales_prj.gold.products').display()


In [0]:
df_dim_cust = spark.sql('select dimcustomerkey,customer_id as dimcustomer_id from sales_prj.gold.customers')

df_dim_prod = spark.sql('select product_id as dimproductkey,product_id as dimproduct_id from sales_prj.gold.products')

In [0]:
df_fact = df.join(df_dim_cust,df.customer_id == df_dim_cust.dimcustomer_id,'left').join(df_dim_prod,df.product_id == df_dim_prod.dimproduct_id,'left')
df_fact.display()

In [0]:
df_fact_new = df_fact.drop('dimcustomer_id','dimproduct_id','customer_id','product_id')
df_fact_new.display()

In [0]:
if spark.catalog.tableExists('sales_prj.gold.fact_sales'):
    dlt_table = DeltaTable.forName('sales_prj','gold.fact_sales')
    dlt_table.alias('t').merge(df_fact_new.alias('s'),'t.order_id = s.order_id and t.dimcustomerkey = s.dimcustomerkey and t.dimproductkey = s.dimproductkey' ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:

    df_fact_new.write.format('delta').mode('append').option('path','abfss://gold@adls5485.dfs.core.windows.net/fact_sales').saveAsTable('sales_prj.gold.fact_sales')

In [0]:
%sql
select * from sales_prj.gold.fact_sales